# Create monthly CESM sea ice dust and black carbon flux files

Interpolated to ANHA12 model grid.

NCAR CESM output:
Community Earth System Model (Community Atmosphere Model - CAM https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1002/2013MS000279) output: https://www.earthsystemgrid.org/; CESM2 Large Ensemble Sea Ice Post Processed Data, Monthly Averages.

Specific run output: https://www.earthsystemgrid.org/dataset/ucar.cgd.cesm2le.output.html

In [4]:
import netCDF4 as nc
import numpy as np
import datetime
import pandas as pd
import datetime as dt
import xarray as xr
import sys
sys.path.append('../paper-materials/')
import mapping_functions as mf
from constants import imin, imax, jmin, jmax

%matplotlib inline

#### Parameters

In [5]:
data_folder = '/data/brogalla/NCAR-CESM2/'

#### Load files

In [6]:
# ANHA12 grid mesh:
mesh        = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask       = mesh['tmask'].values[0,:,:,:]
ANHA12_lons = mesh['nav_lon']
ANHA12_lats = mesh['nav_lat']
Z_masked    = np.ma.masked_where((tmask > 0.1), tmask) 

In [ ]:
# ---- Load data: ------------
# CESM2 files accessed through the Climate Data Gateway
# Black carbon:
BC1 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.SEAICE_BLACK_CARBON_FLUX_CPL.200001-200912.nc')
BC2 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.SEAICE_BLACK_CARBON_FLUX_CPL.201001-201412.nc')
BC3 = xr.open_dataset(f'{data_folder}b.e21.BSSP370cmip6.f09_g17.LE2-1001.001.pop.h.SEAICE_BLACK_CARBON_FLUX_CPL.201501-202412.nc')
ICE_BC1  = BC1['SEAICE_BLACK_CARBON_FLUX_CPL'].values # time, lat, lon; g/cm2/s
ICE_BC2  = BC2['SEAICE_BLACK_CARBON_FLUX_CPL'].values
ICE_BC3  = BC3['SEAICE_BLACK_CARBON_FLUX_CPL'].values
BC1_time = BC1.indexes['time'].to_datetimeindex() 
BC2_time = BC2.indexes['time'].to_datetimeindex() 
BC3_time = BC3.indexes['time'].to_datetimeindex() 
# combine:
ICE_BC   = np.vstack([ICE_BC1, ICE_BC2, ICE_BC3])
BC_time  = np.hstack([BC1_time, BC2_time, BC3_time])
BC_date  = np.array([pd.Timestamp(time).to_pydatetime() for time in BC_time])
  
# Dust:
DST1 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.SEAICE_DUST_FLUX_CPL.200001-200912.nc')
DST2 = xr.open_dataset(f'{data_folder}b.e21.BHISTcmip6.f09_g17.LE2-1001.001.pop.h.SEAICE_DUST_FLUX_CPL.201001-201412.nc')
DST3 = xr.open_dataset(f'{data_folder}b.e21.BSSP370cmip6.f09_g17.LE2-1001.001.pop.h.SEAICE_DUST_FLUX_CPL.201501-202412.nc')
ICE_DST1  = DST1['SEAICE_DUST_FLUX_CPL'].values # time, lat, lon; g/cm2/s
ICE_DST2  = DST2['SEAICE_DUST_FLUX_CPL'].values
ICE_DST3  = DST3['SEAICE_DUST_FLUX_CPL'].values
DST1_time = DST1.indexes['time'].to_datetimeindex() 
DST2_time = DST2.indexes['time'].to_datetimeindex() 
DST3_time = DST3.indexes['time'].to_datetimeindex()
# combine:
ICE_DST   = np.vstack([ICE_DST1, ICE_DST2, ICE_DST3])
DST_time  = np.hstack([DST1_time, DST2_time, DST3_time])
DST_date  = np.array([pd.Timestamp(time).to_pydatetime() for time in DST_time])
    
# Latitudes and longitudes of T grid points
CESM2_lons = BC1['TLONG'].values
CESM2_lons[CESM2_lons >=180] = -360 + CESM2_lons[CESM2_lons >=180] # degrees east
CESM2_lats = BC1['TLAT'].values

#### Functions

In [9]:
# save dust and black carbond forcing fields to a netCDF file:
def save_file(filename, field1, field2):
    ncd = nc.Dataset(filename, 'w', zlib=True)
    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # convert units from g/cm2/s to kg/m2/s:
    field1_convert = field1*10
    field2_convert = field2*10
    
    # variables
    dust             = ncd.createVariable('dust', 'float64', ('y','x'))
    dust.units       = 'kg/m2/s'
    dust.long_name   = 'Dust deposition flux from ice'  
    dust.coordinates = 'nav_lon nav_lat'
    dust[:]          = field1_convert
    
    black_c             = ncd.createVariable('bc', 'float64', ('y','x'))
    black_c.units       = 'kg/m2/s'
    black_c.long_name   = 'Black carbon deposition flux from ice'  
    black_c.coordinates = 'nav_lon nav_lat'
    black_c[:]          = field2_convert
    
    print('saved ', filename)

    ncd.close()
    return

In [10]:
# Identify the indices associated with the specified file year
def find_dates(file_year):
    
    indices = np.array([])
    for d,date in enumerate(DST_date):
        if date.year==file_year:
            indices = np.append(indices, d)
            
    start_index = int(np.amin(indices))
    end_index   = int(np.amax(indices))
    
    print('start index: ', start_index)
    print('end index: ', end_index)
    
    return start_index, end_index

In [11]:
# Main function to interpolate CESM2 fields to the ANHA12 model grid for each year
def CESM_to_ANHA12(file_year, savefiles=False):
    
    si, ei = find_dates(file_year)
    
    dust         = ICE_DST[si:ei+1,:,:]
    black_carbon = ICE_BC[si:ei+1,:,:]
    print('Check that output is of the correct dimensions: ', dust.shape, black_carbon.shape)
    
    interp_dust         = np.empty((12, 2400, 1632))
    interp_black_carbon = np.empty((12, 2400, 1632))
    # loop over the months:
    for i in range(0,12):
        # Mask NaN values and remove
        dstm = np.ma.masked_where(np.isnan(dust[i,:,:]), dust[i,:,:])
        bcm  = np.ma.masked_where(np.isnan(black_carbon[i,:,:]), black_carbon[i,:,:])
        
        interp_dust[i,:,:]         = mf.interp_np(CESM2_lons[~dstm.mask], CESM2_lats[~dstm.mask], \
                                                  dstm[~dstm.mask], ANHA12_lons, ANHA12_lats)
        interp_black_carbon[i,:,:] = mf.interp_np(CESM2_lons[~bcm.mask], CESM2_lats[~bcm.mask], \
                                                  bcm[~bcm.mask], ANHA12_lons, ANHA12_lats)
        
    if savefiles:
        location = '/ocean/brogalla/GEOTRACES/data/Pb-forcing-202311/seaice/'
        
        for ind in range(1,13):
            save_file(f'{location}ice_flux_y{file_year}m{ind:02}.nc',interp_dust[ind-1,:,:],\
                      interp_black_carbon[ind-1,:,:])
    
    return interp_dust, interp_black_carbon

#### Calculations

In [ ]:
for year in np.arange(2002,2022,1):
    interp_dust, interp_black_carbon = CESM_to_ANHA12(year, savefiles=True)